In [23]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import wordninja
from wordninja import LanguageModel
import re
import numpy as np
import gzip
import shutil

def preprocess(x):
    x = str(x)
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 換行符號去除
    return str.strip(x) # 移除左右空白

In [24]:
x = 'xxxapplexxx'
y = 'apple'
x = x[:x.find(y)] + ' ' + y + ' ' + x[x.find(y)+len(y):]
x

'xxx apple xxx'

In [25]:
#df1 = pd.read_excel('../data/台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl').iloc[:,:-1]
#df2 = pd.read_excel('../data/寶典.v3.台塑網.20210901.xlsx',engine='openpyxl')
#df2.columns = df1.columns
#df = df1.append(df2)
df = pd.read_excel('../data/寶典/寶典.v4.20211001.xlsx',engine='openpyxl')
df = df.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df['品名'] = df['品名'].apply(lambda x:x.strip())

# 載入產品csv
try:
    val_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv',index_col=0)[['string_X_train','Y_label','EXPNO','from']].dropna(axis=0)
except:
    val_df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)[['45A','SPEC','EXPNO','from']].dropna(subset=['45A'],axis=0)
    val_df.columns = ['string_X_train','Y_label','EXPNO','from']

# 合併寶典和SPEC
產品集合 = set(df['品名'].values.tolist() + val_df['Y_label'].values.tolist())
產品集合 = set(val_df['Y_label'].values.tolist())
寶典 = [ preprocess(i) for i in 產品集合]
寶典 = sorted(寶典,key=len)[::-1]
寶典[:5]

['VISCOSE STAPLE FIBER 125D X 38MM BRIGHT HIGH TENACITY',
 'ETHYLENEPROPYLENE COPOLYMERSPP COPOLYMER YUNGSOX 3204',
 'VISCOSE STAPLE FIBER 125D X 44MM BRIGHTHIGH TENACITY',
 'VISCOSE STAPLE FIBER 125DX38MM BRIGHT HIGH TENACITY',
 'VISCOSE RAYON STAPLE FIBER15DX40MM SEMI DULL']

# 資料讀取

In [26]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)
print(df.shape)
df = df.rename(columns={'45A':'string_X_train','SPEC':'Y_label'})
df = df[['string_X_train','Y_label','EXPNO','from']].dropna(subset=['string_X_train'],axis=0)
print(df.shape)
df['from'] = df['from'].apply(lambda x:''.join(x.split('-')[1:]))
df['string_X_train'] = df['string_X_train'].apply(preprocess)
df['Y_label'] = df['Y_label'].apply(lambda x:str(x).split('\n'))
df['p'] = 'not find'
for idx in tqdm(df.index):
    for p in df.loc[idx,'Y_label'] + 寶典:
        if p in df.loc[idx,'string_X_train']:
            if (p != '')&(p != ' '):
                df.loc[idx,'p'] = p
                continue
print(df[df['p']!='not find'].shape)
df = df[df['p']!='not find']
df['Y_label'] = df['p']
df = df.drop(['p'],axis=1)

for i in tqdm(df.index):
    x,y = df.loc[i,'string_X_train'],df.loc[i,'Y_label']
    df.loc[i,'string_X_train'] = x[:x.find(y)] + ' ' + y + ' ' + x[x.find(y)+len(y):]
print(df.shape)
df.head(10)

(20737, 17)
(15273, 4)


<ipython-input-26-40ec757e9d13>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm(df.index):



(11571, 5)


<ipython-input-26-40ec757e9d13>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):



(11571, 4)


,string_X_train,Y_label,EXPNO,from
0,SHIPMENT OFPVC SUSPENSION RESIN S65D QTY 18 MT...,PVC MASS RESIN B57,NaN,20210103.xlsx
2,TERMS OF SALE CIF NHAVA SHEVA PORT INDIA70 MT ...,PVC RESIN S60,NaN,20210103.xlsx
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 US...,PVC RESIN B57,11,20210103.xlsx
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,20210103.xlsx
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,20210103.xlsx
6,TERM OF SALE CIF MUNDRA SEAPORT INDIA70 MT OF ...,PVC MASS RESIN B57,NaN,20210103.xlsx
7,COMMODITY ...,YARN,25,20210103.xlsx
8,COMMODITY ...,YARN,25,20210103.xlsx
9,COMMODITY ...,YARN,25,20210103.xlsx
10,COMMODITY ...,YARN,25,20210103.xlsx


In [27]:
# 起始結束位置標註
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst,ye_lst = [],[]
for i in range(len(df)):
    ys,ye = str2index(df['string_X_train'].values[i],df['Y_label'].values[i])
    ys_lst.append(ys),ye_lst.append(ye)
    
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
print(1,df.shape)

df = df[df['Y_label']!=''] # 去掉空值
print(2,df.shape)

df = df.loc[df['string_Y_1']!=-1,:] # 去掉找不到答案的
print(3,df.shape)

df = df.dropna(subset=['string_X_train','Y_label'],axis=0) # 去掉NAN
print(4,df.shape)

df.head(10)

1 (11571, 6)
2 (11571, 6)
3 (11571, 6)
4 (11571, 6)


,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
0,SHIPMENT OFPVC SUSPENSION RESIN S65D QTY 18 MT...,PVC MASS RESIN B57,NaN,20210103.xlsx,81,99
2,TERMS OF SALE CIF NHAVA SHEVA PORT INDIA70 MT ...,PVC RESIN S60,NaN,20210103.xlsx,50,63
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 US...,PVC RESIN B57,11,20210103.xlsx,6,19
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,20210103.xlsx,1,19
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,20210103.xlsx,17,28
6,TERM OF SALE CIF MUNDRA SEAPORT INDIA70 MT OF ...,PVC MASS RESIN B57,NaN,20210103.xlsx,98,116
7,COMMODITY ...,YARN,25,20210103.xlsx,85,89
8,COMMODITY ...,YARN,25,20210103.xlsx,85,89
9,COMMODITY ...,YARN,25,20210103.xlsx,85,89
10,COMMODITY ...,YARN,25,20210103.xlsx,85,89


In [28]:
# 最後檢查 Y1 Y2 位置 和 Y_LABEL要對應上
for i in tqdm(df.index):
    a = df.loc[i,'string_X_train'][df.loc[i,'string_Y_1']:df.loc[i,'string_Y_2']]
    b = df.loc[i,'Y_label']
    assert a==b

<ipython-input-28-4ec65d08a61a>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


In [29]:
from termcolor import colored

# 實際感受一下
def str2index(context,string):
    if type(string) != str:
        print(string)
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

def color_output(text_input,text_output):
    ys,ye = str2index(text_input,text_output)
    left = text_input[:ys]
    mid = text_output
    right = text_input[ye:]
    print(left,colored(mid,'red'),right)

for j in range(10):
    i = np.random.randint(low=0,high=len(df))
    x = df['string_X_train'].values[i]
    p = df['Y_label'].values[i]
    print(j)
    color_output(x,p)
    print(colored(p,'green'))

0
CIF HONG KONGPROPYLENE COPOLYMER   GRADE   NOK8802QUANTITY2761MTUNIT PRICEUSD980MTQUANTITY439MTUNIT PRICEUSD1130MT
GRADE
1
1ABSGF RESIN   GRADE   NOAG20GF BLACK4MTUSD2350MT2PCGF RESIN GRADE NOAC2120 BLACK13MTUSD2630MT3PCGF RESIN GRADE NOAC2120 BLACK4MTUSD2630MT4PP RESIN GRADE NOK453524MTUSD1490MT5ABSGF RESIN GRADE NOAG20GF BLUE3MTUSD2950MT6PP RESIN GRADE NOC42201MTUSD2700MTITEM 1AND2CIF ANY CHINA PORTITEM 3AND4AND5AND6CIP ANY CHINA PORTHONG KONG
GRADE
2
1COMMODITYHDPE   TAISOX 9001  QUANTITY600MT  UNIT PRICEUSD1215MTTOTAL PRICESUSD729000PACKINGSTANDARD EXPORT PACKINGORIGINTAIWANPRICE TERMCIF CHINA MAIN PORTREF NOH0D1042COMMODITYHDPE TAISOX 8100RTQUANTITY300MT  UNIT PRICEUSD1380MTTOTAL PRICESUSD414000PACKINGSTANDARD EXPORT PACKINGORIGINTAIWANPRICE TERMCIF CHINA MAIN PORTSC NOH0DW10
TAISOX 9001
3
PURIFIED ISOPHTHALIC   ACID             360 MTSCIF NHAVA SHEVA PORT INDIA
ACID
4
COMMODITY  PURIFIED TEREPHTHALIC   ACID   PTA  QUANTITY 144000 MTS  UNIT PRICE USD435MT  TRADE TERMS CIF CAT LA

In [30]:
# 保存
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_產品.csv')
df

(11571, 6)


,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
0,SHIPMENT OFPVC SUSPENSION RESIN S65D QTY 18 MT...,PVC MASS RESIN B57,NaN,20210103.xlsx,81,99
2,TERMS OF SALE CIF NHAVA SHEVA PORT INDIA70 MT ...,PVC RESIN S60,NaN,20210103.xlsx,50,63
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 US...,PVC RESIN B57,11,20210103.xlsx,6,19
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,20210103.xlsx,1,19
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,20210103.xlsx,17,28
...,...,...,...,...,...,...
20732,PVC SUSPENSION RESIN S65D FOR 70MT AT USD 12...,PVC SUSPENSION RESIN,NaN,20200712.xlsx,1,21
20733,7000 MT IN 4X20 FT CONTAINERS PVC RESIN S60 AS...,PVC RESIN S60,11,20200712.xlsx,30,43
20734,EPOXY RESIN NPEF178FOB ANY PORT OF TAIWAN,EPOXY RESIN,NaN,20200712.xlsx,1,12
20735,EPOXY RESIN NPEL128CIF SAVANNAH GA,EPOXY RESIN,NaN,20200712.xlsx,1,12
